In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [99]:
df = pd.read_csv("Data/clean.csv")

In [100]:
df.columns

Index(['Year', 'Mo', 'Dy', 'Tsu', 'Eq', 'Name', 'Loc', 'Ctry', 'Lat', 'Lon',
       'Ele', 'Type', 'VEI', 'Agent', 'Deaths', 'Death_des', 'Missing',
       'Missing_des', 'Injuries', 'Injuries_des', 'Dmg', 'Dmg_des', 'Houses',
       'Houses_des', 'Death_tot', 'Death_tot_des', 'Missing_tot',
       'Missing_tot_des', 'Injuries_tot', 'Injuries_tot_des', 'Dmg_tot',
       'Dmg_tot_des', 'Houses_tot', 'Houses_tot_des'],
      dtype='object')

In [101]:
agent_df = df[((df["Death_tot_des"].notna()) | (df["Death_tot"] > 0)) & (df["Agent"].notna())][["VEI", "Agent", "Death_tot", "Death_tot_des"]]
agent_df

,VEI,Agent,Death_tot,Death_tot_des
1,7.0,P,NaN,3.0
3,6.0,T,NaN,1.0
13,7.0,W,NaN,3.0
19,4.0,M,NaN,1.0
29,5.0,"P,T",2100.0,4.0
...,...,...,...,...
861,2.0,"L,I,S",32.0,1.0
862,NaN,L,1.0,1.0
863,3.0,T,45.0,1.0
865,5.0,W,6.0,1.0


In [102]:
agent_df.loc[agent_df[(agent_df["Death_tot_des"] == 1) & (agent_df["Death_tot"].isna())].index, "Death_tot"] = np.round(np.e ** ((np.log(1) + np.log(50)) / 2))

agent_df.loc[agent_df[(agent_df["Death_tot_des"] == 2) & (agent_df["Death_tot"].isna())].index, "Death_tot"] = np.round(np.e ** ((np.log(51) + np.log(100)) / 2))

agent_df.loc[agent_df[(agent_df["Death_tot_des"] == 3) & (agent_df["Death_tot"].isna())].index, "Death_tot"] = np.round(np.e ** ((np.log(100) + np.log(1000)) / 2))

agent_df.loc[agent_df[(agent_df["Death_tot_des"] == 4) & (agent_df["Death_tot"].isna())].index, "Death_tot"] = np.round(np.e ** ((np.log(1000) + np.log(10000)) / 2))

In [103]:
agent_df

,VEI,Agent,Death_tot,Death_tot_des
1,7.0,P,316.0,3.0
3,6.0,T,7.0,1.0
13,7.0,W,316.0,3.0
19,4.0,M,7.0,1.0
29,5.0,"P,T",2100.0,4.0
...,...,...,...,...
861,2.0,"L,I,S",32.0,1.0
862,NaN,L,1.0,1.0
863,3.0,T,45.0,1.0
865,5.0,W,6.0,1.0


In [104]:
agent_df["Death_tot"].isna().sum()

0

In [105]:
agent_df.dropna(inplace=True)

In [106]:
agent_df.drop(columns="Death_tot_des", inplace=True)

In [107]:
agent_df

,VEI,Agent,Death_tot
1,7.0,P,316.0
3,6.0,T,7.0
13,7.0,W,316.0
19,4.0,M,7.0
29,5.0,"P,T",2100.0
...,...,...,...
856,4.0,T,39.0
861,2.0,"L,I,S",32.0
863,3.0,T,45.0
865,5.0,W,6.0


Split according to agent

In [108]:
(agent_df[agent_df["Agent"].str.contains('T')]["Death_tot"] / ((agent_df[agent_df["Agent"].str.contains('T')]["Agent"].str.len() + 1) // 2)).sum()

11764.249999999998

In [109]:
agent_df[agent_df["Agent"].str.contains('T')]

,VEI,Agent,Death_tot
3,6.0,T,7.0
29,5.0,"P,T",2100.0
38,2.0,T,7.0
42,4.0,"T,L",80.0
72,5.0,"T,F",220.0
...,...,...,...
840,3.0,"T,G,M",4.0
853,2.0,"P,T",1.0
855,2.0,T,22.0
856,4.0,T,39.0


In [123]:
letter_to_agent = {
"A" : "Avalanche",
"E" : "Electrical (Lightning)",
"F" : "Floods",
"G" : "Gas",
"I" : "Indirect deaths (disease, starvation)",
"L" : "Lava flows",
"M" : "Mudflows",
"m" : "Secondary (post-eruption) mudflows",
"P" : "Pyroclastic flows, surges, & direct blasts",
"S" : "Seismic, or volcanic earthquake",
"T" : "Tephra (ash, bombs, steam blasts)",
"W" : "Waves or tsunami",
}

In [124]:
agent_dict = [{'Agent': letter_to_agent[letter],
  'VEI': i,
  'Death_tot': np.round((agent_df[(agent_df["Agent"].str.contains(letter)) | (agent_df["VEI"] == i)]["Death_tot"] / ((agent_df[(agent_df["Agent"].str.contains(letter)) | (agent_df["VEI"] == i)]["Agent"].str.len() + 1) // 2)).sum()) } for letter in letter_to_agent.keys() for i in range(1, 8) ]

agent_df_new = pd.DataFrame.from_dict(agent_dict)

In [125]:
agent_df_new

,Agent,VEI,Death_tot
0,Avalanche,1,28479.0
1,Avalanche,2,35378.0
2,Avalanche,3,76022.0
3,Avalanche,4,89293.0
4,Avalanche,5,32980.0
...,...,...,...
79,Waves or tsunami,3,94357.0
80,Waves or tsunami,4,104621.0
81,Waves or tsunami,5,48734.0
82,Waves or tsunami,6,48021.0


In [126]:
agent_df_new.to_csv("Data/agents_VEI_deaths_2.csv", index=False)

In [127]:
df.__len__()

868